# Single Flight Simulation - Complete Workflow

This notebook demonstrates a detailed single flight simulation workflow.

## Topics Covered

- Configuration management
- Component building and validation
- Flight simulation execution
- Comprehensive result analysis
- Data export in multiple formats
- Visualization suite

In [ ]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt

sys.path.insert(0, str(Path.cwd().parent / 'src'))

from config_loader import ConfigLoader
from motor_builder import MotorBuilder
from environment_setup import EnvironmentBuilder
from rocket_builder import RocketBuilder
from flight_simulator import FlightSimulator
from visualizer import Visualizer
from data_handler import DataHandler
from validators import RocketValidator, MotorValidator, EnvironmentValidator

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## Part 1: Load and Validate Configuration

In [ ]:
config_path = Path('../configs/complete_example.yaml')

config_loader = ConfigLoader()
config_loader.load_from_yaml(str(config_path))

rocket_cfg = config_loader.get_rocket_config()
motor_cfg = config_loader.get_motor_config()
env_cfg = config_loader.get_environment_config()
sim_cfg = config_loader.get_simulation_config()

print(f"Configuration loaded: {rocket_cfg.name}")

In [ ]:
# Validate configurations
rocket_warnings = RocketValidator.validate(rocket_cfg)
motor_warnings = MotorValidator.validate(motor_cfg)
env_warnings = EnvironmentValidator.validate(env_cfg)

print("Validation complete:")
print(f"  Rocket warnings: {len(rocket_warnings)}")
print(f"  Motor warnings: {len(motor_warnings)}")
print(f"  Environment warnings: {len(env_warnings)}")

for warning in rocket_warnings + motor_warnings + env_warnings:
    print(f"  ⚠ {warning.message}")

## Part 2: Build Motor

In [ ]:
motor_builder = MotorBuilder(motor_cfg)
motor = motor_builder.build()

print(motor_builder.get_summary())

## Part 3: Build Environment

In [ ]:
env_builder = EnvironmentBuilder(env_cfg)
environment = env_builder.build()

print(env_builder.get_summary())

## Part 4: Build Rocket

In [ ]:
rocket_builder = RocketBuilder(rocket_cfg)
rocket = rocket_builder.build(motor)

print(rocket_builder.get_summary())

stability = rocket_builder.get_stability_info()
print(f"\nStability Margin: {stability['static_margin_calibers']:.2f} calibers")
print(f"Status: {'✓ STABLE' if stability['static_margin_calibers'] > 1 else '✗ UNSTABLE'}")

## Part 5: Run Flight Simulation

In [ ]:
simulator = FlightSimulator(
    rocket=rocket,
    motor=motor,
    environment=environment,
    rail_length_m=sim_cfg.rail_length_m,
    inclination_deg=sim_cfg.inclination_deg,
    heading_deg=sim_cfg.heading_deg
)

flight = simulator.run()
simulator.print_summary()

## Part 6: Generate All Visualizations

In [ ]:
visualizer = Visualizer(flight)
output_dir = Path('../outputs/single_flight')

visualizer.create_standard_plots(str(output_dir))
print(f"All plots saved to {output_dir}")

## Part 7: Export Data

In [ ]:
data_handler = DataHandler()

# Export everything
summary = simulator.get_summary()

data_handler.export_complete_dataset(
    flight=flight,
    summary=summary,
    output_dir=str(output_dir),
    formats=['csv', 'json']
)

print(f"Data exported to {output_dir}")